In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from keras import layers
from tensorflow.keras import utils

import pathlib

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "/content/train-data.tsv"
test_file_path = "/content/valid-data.tsv"

In [ ]:
# CREATING A TRAINING AND TESTING DATASETS
train_data=pd.read_csv(train_file_path, sep='\t',names=["type","message"])
test_data=pd.read_csv(test_file_path, sep='\t',names=["type","message"])

train_features=train_data.copy()
train_labels=train_features.pop('type')
test_features=test_data.copy()
test_labels=test_features.pop('type')

train_labels=pd.get_dummies(train_labels)
test_labels=pd.get_dummies(test_labels)
train_labels=train_labels.drop(['spam'],axis=1)
test_labels=test_labels.drop(['spam'],axis=1)

train_features=train_features.to_numpy().flatten()
test_features=test_features.to_numpy().flatten()
train_labels=train_labels.to_numpy().flatten()
test_labels=test_labels.to_numpy().flatten()

In [ ]:
# ENCODER FOR TEXT VECTORIZATION
VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_features)

In [ ]:
# CREATING THE MODEL
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
# COMPILING THE MODEL
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
# TRAINING THE MODEL
history = model.fit(x=train_features,y=train_labels, epochs=10,validation_split=0.2,
                    validation_steps=30)

In [ ]:
# TEST LOSS AND ACCURACY
test_loss, test_acc = model.evaluate(x=test_features,y=test_labels)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

In [ ]:
# TESTING PREDICTIONS MANUALLY
sample_text = ("wow, is your arm alright. that happened to me one time too")
predictions = model.predict([sample_text])

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

  sample_text = (pred_text)
  predictions = model.predict([sample_text])
  prediction=[predictions[0][0],'ham' if predictions[0][0]>0.8 else 'spam']
  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
